In [1]:
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")
model.cuda()
model.eval();

/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

Deformable Convolution not built!
Deformable Convolution not built!
Use HM Bias:  -2.19


In [2]:
import pickle
pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
        
with open(pickle_dir + "findcenter_output1.pkl", 'rb') as handle:
    findcenter_output1 = pickle.load(handle)
with open(pickle_dir + "findcenter_output2.pkl", 'rb') as handle:
    findcenter_output2 = pickle.load(handle)
with open(pickle_dir + "findcenter_output3.pkl", 'rb') as handle:
    findcenter_output3 = pickle.load(handle)

In [3]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = model.neck.find_centers(x)
    
print((ct_feat==findcenter_output1).all())
print((center_pos_embedding==findcenter_output2).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [4]:
for i in range(out_scores.shape[0]):
    result = (out_scores[i]==findcenter_output3[i]['scores']).all()
    result = result and (out_labels[i]==findcenter_output3[i]['labels']).all()
    result = result and (out_orders[i]==findcenter_output3[i]['order']).all()
    print(result)

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


In [5]:
import tensorrt as trt
trt.__version__

'8.5.1.7'

In [6]:
        
def load_engine(engine_filepath):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with open(engine_filepath, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine

centerFinder_engine_path = 'findCenter_folded.trt'
cf_engine = load_engine(centerFinder_engine_path)
cf_context = cf_engine.create_execution_context()

[12/13/2023-14:56:13] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


In [7]:
with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    input_tensor = pickle.load(handle)
print((x == input_tensor).all(), x.device)

tensor(True, device='cuda:0') cuda:0


In [8]:
ct_feat_tensor = torch.zeros((1, 3000, 256), dtype=torch.float32, device=x.device)
center_pos_embedding_tensor = torch.zeros((1, 3000, 256), dtype=torch.float32, device=x.device)
out_scores_tensor = torch.zeros((6, 1, 500), dtype=torch.float32, device=x.device)
out_labels_tensor = torch.zeros((6, 1, 500), dtype=torch.int32, device=x.device)
out_orders_tensor = torch.zeros((6, 1, 500), dtype=torch.int32, device=x.device)
out_masks_tensor = torch.zeros((6, 1, 500), dtype=torch.bool, device=x.device)
IO_tensors = {
    "inputs" :
    {'input_tensor': x},
    "outputs" :
    {'ct_feat': ct_feat_tensor, 'center_pos_embedding': center_pos_embedding_tensor,
    'out_scores': out_scores_tensor, 'out_labels': out_labels_tensor,
    'out_orders': out_orders_tensor, 'out_masks': out_masks_tensor}
}

In [9]:
def run_trt_engine(context, engine, tensors):
    bindings = [None]*engine.num_bindings
    for idx, binding in enumerate(engine):
        tensor_name = engine.get_tensor_name(idx)
        if engine.get_tensor_mode(binding)==trt.TensorIOMode.INPUT:
            bindings[idx] = tensors['inputs'][tensor_name].data_ptr()
            if context.get_tensor_shape(tensor_name):
                context.set_input_shape(tensor_name, tensors['inputs'][tensor_name].shape)
        else:
            bindings[idx] = tensors['outputs'][tensor_name].data_ptr()
    context.execute_v2(bindings=bindings)

run_trt_engine(cf_context, cf_engine, IO_tensors)

In [10]:
print((out_scores == out_scores_tensor).all())
print((out_labels == out_labels_tensor).all())
print((out_orders == out_orders_tensor).all())
print((out_masks == out_masks_tensor).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


In [11]:
print(ct_feat[0][0][:20])
print(ct_feat_tensor[0][0][:20])

tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7531, 0.0000, 0.0000,
        0.0000, 0.8260], device='cuda:0')
tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7531, 0.0000, 0.0000,
        0.0000, 0.8260], device='cuda:0')


In [12]:
with torch.no_grad():
    out_dict_lists = model.neck.forward(x)
    out_dict_lists_trt = model.neck.forward_trt(x)

[12/13/2023-14:56:13] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
ct_feat2
tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349], device='cuda:0')
tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349], device='cuda:0')


AssertionError: 

In [ ]:
out_dict_lists_trt[0].keys()

In [ ]:
print(out_dict_lists[0]['ct_feat'].dtype, out_dict_lists_trt[0]['ct_feat'].dtype)
print(out_dict_lists[0]['scores'].dtype, out_dict_lists_trt[0]['scores'].dtype)
print(out_dict_lists[0]['labels'].dtype, out_dict_lists_trt[0]['labels'].dtype)
print(out_dict_lists[0]['order'].dtype, out_dict_lists_trt[0]['order'].dtype)
print(out_dict_lists[0]['mask'].dtype, out_dict_lists_trt[0]['mask'].dtype)


In [ ]:
for i in range(len(out_dict_lists_trt)):
    result = (out_dict_lists[i]['scores'] == out_dict_lists_trt[i]['scores']).all()
    result = result and (out_dict_lists[i]['labels'] == out_dict_lists_trt[i]['labels']).all()
    result = result and (out_dict_lists[i]['order'] == out_dict_lists_trt[i]['order']).all()
    result = result and (out_dict_lists[i]['mask'] == out_dict_lists_trt[i]['mask']).all()
    print(result)

In [ ]:
print(out_dict_lists[0]['ct_feat'][0][0][:20])
print(out_dict_lists_trt[0]['ct_feat'][0][0][:20])